<a href="https://colab.research.google.com/github/pstorniolo/Master2021/blob/main/2021_11_06_1_SparkNLP_SparkML_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Colab** setup

In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip -q install --ignore-installed pyspark==3.0.3

# Install Spark NLP
! pip -q install --ignore-installed spark-nlp==3.1.0

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

https://it.wikipedia.org/wiki/John_Snow_(medico)

https://www.johnsnowlabs.com/spark-nlp/
#Spark NLP

**Spark ML** fornisce un set di applicazioni di Machine Learning che possono essere predisposte utilizzando due componenti principali: *Estimators* e *Transformers*. Gli *Estimators* hanno un metodo chiamato ***fit()*** che protegge e addestra una parte di dati a tale applicazione. Il *Transformer* è generalmente il risultato di un processo di adattamento e applica modifiche al set di dati di destinazione. Questi componenti sono stati incorporati per essere utilizzabili da **Spark NLP**.

Le **Pipeline** sono un meccanismo per combinare più *Estimators* e *Transformers* in un unico flusso di lavoro. Consentono più trasformazioni concatenate lungo un'attività di Machine Learning.

https://nlp.johnsnowlabs.com/api/python/

##Annotation
Il risultato di base di un'operazione **Spark NLP** è un'annotazione. La sua struttura comprende:
*   *annotatorType*: il tipo di annotatore che ha generato l'annotazione corrente
*   *begin*: l'inizio del contenuto corrispondente relativo al testo non elaborato
*   *end*: la fine del contenuto corrispondente relativo al testo non elaborato
*   *result*: l'output principale dell'annotazione
*   *metadata*: contenuto del risultato abbinato e informazioni aggiuntive
*   *embedding*: contiene mappature vettoriali se necessarie

Questo oggetto viene generato automaticamente dagli annotatori dopo un processo di trasformazione. Non è richiesto alcun lavoro manuale.

###*Annotators*
Gli *Annotators* sono la punta di diamante delle funzioni NLP in Spark NLP. Esistono due forme di annotatori:
*   *Annotator Approaches*: sono quelli che rappresentano uno Spark ML Estimator e richiedono una fase di formazione. Hanno una funzione chiamata fit(data) che addestra un modello basato su alcuni dati. Producono il secondo tipo di annotatore che è un modello di annotatore o trasformatore.
*   *Annotator Models*: sono modelli spark o *Trasformators*, nel senso che hanno una funzione di trasformazione (dati). Questa funzione prende come input un dataframe a cui aggiunge una nuova colonna contenente il risultato dell'annotazione corrente. Tutti i *Trasformators* sono additivi, nel senso che si aggiungono ai dati correnti, non sostituiscono o eliminano mai le informazioni precedenti.

Entrambe le forme di annotatori possono essere incluse in una pipeline. Tutti gli annotatori inclusi in una pipeline verranno eseguiti automaticamente nell'ordine definito e trasformeranno i dati di conseguenza. Una Pipeline viene trasformata in un *PipelineModel* dopo la fase *fit()*. La *Pipeline* può essere salvata su disco e ricaricata in qualsiasi momento.

###*Common Functions*
*   *setInputCols(column_names)*: prende un elenco di nomi di colonne di annotazioni richieste da questo annotatore. Questi sono generati dagli annotatori che precedono l'annotatore corrente nella pipeline.
*   *setOutputCol(column_name)*: definisce il nome della colonna contenente il risultato dell'annotatore corrente. Usa questo nome come input per altri annotatori lungo la pipeline che richiedono gli output generati dall'annotatore corrente.

##**Pretrained** pipeline
**Explain Document ML** (*explain_document_ml*) è una pipeline preaddestrata che fa un po' di tutto ciò che riguarda la NLP.

https://nlp.johnsnowlabs.com/docs/en/pipelines

https://nlp.johnsnowlabs.com/models

In [ ]:
import sparknlp
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

from sparknlp.pretrained import PretrainedPipeline

In [ ]:
# Download the pretrained pipeline from Johnsnowlab's servers
explain_document_pipeline = PretrainedPipeline("explain_document_ml")

annotations = explain_document_pipeline.annotate("We are very happy about SparkNLP")
print(annotations)

Spark DataFrame

In [ ]:
sentences = [
  ['Hello, this is an example sentence'],
  ['And this is a second sentence.']
]

# spark is the Spark Session automatically started by pyspark.
data = spark.createDataFrame(sentences).toDF("text")

In [ ]:
# Transform 'data' and store output in a new 'annotations_df' dataframe
annotations_df = explain_document_pipeline.transform(data)

In [ ]:
# Show the results
annotations_df.printSchema()
annotations_df.show(truncate=False)

##Manipulating pipelines

In [ ]:
annotations_df.select("token").show(truncate=False)

Per vedere solo delle annotazioni risultanti possiamo utilizzare l'annotatore *Finisher*, recuperare la pipeline *Explain Document ML* e aggiungerli insieme in una pipeline Spark ML. Le pipeline preaddestrate si aspettano che la colonna di input sia denominata "text".

In [ ]:
from sparknlp import Finisher
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline

In [ ]:
explain_pipeline_model = PretrainedPipeline("explain_document_ml").model
finisher = Finisher().setInputCols(["token", "lemmas", "pos"])

pipeline = Pipeline().setStages([
               explain_pipeline_model,
               finisher
           ])

In [ ]:
sentences = [
    ['Hello, this is an example sentence'],
    ['And this is a second sentence.']
]
data = spark.createDataFrame(sentences).toDF("text")

model = pipeline.fit(data)
annotations_finished_df = model.transform(data)

annotations_finished_df.select('finished_token').show(truncate=False)

# Spark NLP and Spark ML Pipelines

## Simple Topic Modeling

`Spark-NLP`
* DocumentAssembler
* SentenceDetector
* Tokenizer
* Normalizer
* POS tagger
* Chunker
* Finisher

`Spark ML`
* Hashing
* [TF-IDF](https://it.wikipedia.org/wiki/Tf-idf)
* [LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation)

In [ ]:
import sys
import time

#Spark ML & SQL
from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, Tokenizer
from pyspark.ml.clustering import LDA, LDAModel
from pyspark.sql.functions import col

#Spark NLP
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import *

### Let's create a Spark Session for our app

In [ ]:
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Let's download some scientific sample from PubMed dataset:
```
wget -N 	https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv -P /tmp
```

In [ ]:
! wget -q -N https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv -P /tmp

In [ ]:
#!cat /tmp/pubmed-sample.csv

In [ ]:
pubMedDF = spark.read.option("header", "true").csv("/tmp/pubmed-sample.csv").filter("AB IS NOT null").withColumn("text", col("AB")).drop("TI", "AB")
pubMedDF.printSchema()
pubMedDF.show(truncate=False)

In [ ]:
print(pubMedDF.count())

pubMedDF = pubMedDF.limit(2000)

### Let's create Spark-NLP Pipeline

In [ ]:
# Spark NLP Pipeline

document_assembler = DocumentAssembler().setInputCol("text")

sentence_detector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")

tokenizer = Tokenizer().setInputCols(["sentence"]).setOutputCol("token")

posTagger = PerceptronModel.pretrained().setInputCols(["sentence", "token"])

chunker = Chunker().setInputCols(["sentence", "pos"]).setOutputCol("chunk").setRegexParsers(["<NNP>+", "<DT>?<JJ>*<NN>"])

finisher = Finisher().setInputCols(["chunk"]).setIncludeMetadata(False)

nlpPipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector, 
    tokenizer,
    posTagger,
    chunker,
    finisher
])

In [ ]:
nlpPipelineDF = nlpPipeline.fit(pubMedDF).transform(pubMedDF)
nlpPipelineDF.printSchema()

### Let's create Spark ML Pipeline

https://spark.apache.org/docs/latest/mllib-feature-extraction.html

*   *TF*: Term Frequency
*   *DF*: Document Frequency
*   *IDF*: Inverse Document Frequency
*   *LDA*: Latent Dirichlet Allocation
*   *CountVectorizer*: Estrae un vocabolario dalle raccolte di documenti e genera un *CountVectorizerModel*





In [ ]:
# SPark ML Pipeline

cv = CountVectorizer(inputCol="finished_chunk", outputCol="features", vocabSize=1000, minDF=10.0, minTF=10.0)
idf = IDF(inputCol="features", outputCol="idf")
lda = LDA(k=10, maxIter=5)

### Let's create Spark-NLP Pipeline
mlPipeline = Pipeline(stages=[
    cv,
    idf,
    lda
])

###We are going to train Spark ML Pipeline by using Spark-NLP Pipeline

In [ ]:
# Let's create Spark-NLP Pipeline
mlModel = mlPipeline.fit(nlpPipelineDF)

In [ ]:
mlPipelineDF = mlModel.transform(nlpPipelineDF)

In [ ]:
mlPipelineDF.printSchema()
mlPipelineDF.show()

https://spark.apache.org/docs/latest/ml-pipeline.html

In [ ]:
ldaModel = mlModel.stages[2]

In [ ]:
ll = ldaModel.logLikelihood(mlPipelineDF)
lp = ldaModel.logPerplexity(mlPipelineDF)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))


In [ ]:
# Describe topics.
print("The topics described by their top-weighted terms (the first three):")
ldaModel.describeTopics(3).show(truncate=False)

### Let's look at out topics
NOTA: più operazioni di *cleaning*, *filtering*, *playing* con `CountVectorizer` e più iterazioni in `LDA` porteranno a migliori risultati di *Topic Modeling*.

https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.CountVectorizer.html

https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.CountVectorizerModel.html

In [ ]:
# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize()) + " words):")

topics = ldaModel.describeTopics(20)
topics_rdd = topics.rdd

vocab = mlModel.stages[0].vocabulary

In [ ]:
topics_words = topics_rdd.map(lambda row: row['termIndices']).map(lambda idx_list: [vocab[idx] for idx in idx_list]).collect()

for idx, topic in enumerate(topics_words):
    print("topic: ", idx)
    print("----------------------------------------")
    for word in topic:
       print(word)
    print("----------------------------------------")